<a href="https://colab.research.google.com/github/danishmahmoodali/Deep-Learning-Projects/blob/main/Classification/Bilingual_Tweet_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bilingual Tweet Classification

Group Members:
1.   Danish Mahmood Ali
2.   Muhammad Ibad
3.   Ali bin Ejaz
4.   Reeba Nadeem



In [ ]:
#Downloading and extracting dataset

#!gdown --id 18F7kqqoOcHpMt8Gwmj95pV7nCyu5xqyh #basic-human labelled data 3000
#!gdown --id 10I-rrFBMKY7fm2hWDkQ6OjNS2B8uuQhD #new_labelled data
#!gdown --id 17VM7krYYRcd2uFYtRrTrpwjxucxYBrRI #generated data

!gdown --id 11LXff9reIVVASkJe8CDm1pyAmuvKOTYi #new_data
!gdown --id 1rptRiR7bmrQYdk6P9a_wsTg6NvsNhur3 #labelled_data(correctly preprocessed)
!gdown --id 19mL8Zq_M6eXyEqQJFmr-AAH1lZ3EKzHs #raw_data

#!unzip '/content/basic-Human labelled 3000 tweets.zip'
#!unzip '/content/new_data.zip'
#!unzip '/content/generated_data(old).zip'

!unzip '/content/raw_data_new.zip'
!unzip '/content/labelled_data.zip'
!unzip '/content/new_basic_human.zip'

!mkdir semisets
!mkdir processed_data

Streaming output truncated to the last 5000 lines.
  inflating: labelled_data/1285@others.txt  
  inflating: labelled_data/1286@politics.txt  
  inflating: labelled_data/1287@politics.txt  
  inflating: labelled_data/1288@currentaffairs.txt  
  inflating: labelled_data/1289@others.txt  
  inflating: labelled_data/128@currentaffairs.txt  
  inflating: labelled_data/128@others.txt  
  inflating: labelled_data/1290@others.txt  
  inflating: labelled_data/1291@currentaffairs.txt  
  inflating: labelled_data/1292@politics.txt  
  inflating: labelled_data/1293@entertainment.txt  
  inflating: labelled_data/1294@others.txt  
  inflating: labelled_data/1295@others.txt  
  inflating: labelled_data/1296@politics.txt  
  inflating: labelled_data/1297@currentaffairs.txt  
  inflating: labelled_data/1298@currentaffairs.txt  
  inflating: labelled_data/1299@others.txt  
  inflating: labelled_data/129@currentaffairs.txt  
  inflating: labelled_data/129@others.txt  
  inflating: labelled_data/12@curre

In [ ]:
#Importing stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#making most common word dictionary

import os
import numpy as np
#import contractions
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB, MultinomialNB
from sklearn import svm
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

def make_di(count):
    direc = "/content/new_basic_human/"
    direc1 = "/content/labelled_data/"
    files = os.listdir(direc) + os.listdir(direc)
    emails = [direc + email for email in files]
    stop_words = set(stopwords.words('english'))
    words = []
    c = len(emails)
    for email in emails:
        f = open(email, encoding="utf8")
        blob = f.read()
        words += blob.split(" ")
       # print(c)
        c -= 1

    #print(words)

    for i in reversed(range(len(words))):
        words[i] = words[i].lower()
        if words[i] in stop_words or not words[i].isalpha() or len(words[i]) < 2 or words[i] == 'amp':
            words.remove(words[i])

    #words = check_similarity(words)
    dictionary = Counter(words)
    return(dictionary.most_common(count))


#d = make_di(3000)
#print(d)

In [ ]:
#make dataset from supervised learning dataset

def make_datasetknn(dictionary):
    direc = "/content/raw_data_new/"
    files = os.listdir(direc)
    emails = [direc + email for email in files]
    words = []
    feature_set = []
    labels = []
    stop_words = set(stopwords.words('english'))
    c = len(emails)
    fileno = 0
    for email in emails:
        f = open(email, encoding="utf8")
        blob = f.read()
        words = blob.split(" ")
        for i in reversed(range(len(words))):
            words[i] = words[i].lower()
            if words[i] in stop_words or not words[i].isalpha() or len(words[i]) < 2 :
                words.remove(words[i])
        f2 = open('/content/semisets/'+str(fileno)+'.txt','w')
        for word in words:
          f2.write(word+' ')


        vector=[]
        singlevec=[]
        for featureword in dictionary:
            w=featureword[0]

            if w in words:
                vector.append(1)

            else:
                  vector.append(0)

        #vector.append(singlevec)
        #f2 = open('/content/semisets/'+str(fileno)+'.txt','w')
        #try:
            #f2.write(words)
        #except:
            #print('exception')
        #f2.close()
        feature_set.append(vector)
        fileno = fileno + 1
        #if "politics" in email:
           # labels.append(0)
           # feature_set.append(vector)
       # if "fun" in email:
         #   labels.append(1)
        #    feature_set.append(vector)
       # if "tech" in email:
         #   labels.append(2)
        #    feature_set.append(vector)
      #  if "curr" in email:
        #    labels.append(3)
        #    feature_set.append(vector)
      #  if "enter" in email:
        #    labels.append(4)
        #    feature_set.append(vector)
       # if "other" in email:
       #     labels.append(5)
       #     feature_set.append(vector)

        #print(c)
        c = c - 1

    return feature_set,fileno #, labels,emails


In [ ]:
from pandas.core.construction import is_empty_data
#make dataset from human labelled data
import collections, numpy

def make_datasetknnbase(dictionary):
    direc = "/content/new_basic_human/"
    direc1 = "/content/labelled_data/"
    files = os.listdir(direc) + os.listdir(direc)
    emails = [direc + email for email in files]
    feature_set = []
    labels = []
    c = len(emails)
    for email in emails:
        f = open(email ,encoding="utf8")
        words = []
        words = f.read().split(' ')
        vector=[]
        singlevec=[]
        for featureword in dictionary:
            w=featureword[0]

            if w in words:
                vector.append(1)
            else:
                vector.append(0)
        #vector.append(singlevec)



        if "politics" in email:
            labels.append(0)
            feature_set.append(vector)
        if "fun" in email:
            labels.append(1)
            feature_set.append(vector)
        if "tech" in email:
            labels.append(2)
            feature_set.append(vector)
        if "curr" in email:
            labels.append(3)
            feature_set.append(vector)
        if "enter" in email:
            labels.append(4)
            feature_set.append(vector)
        if "other" in email:
            labels.append(5)
            feature_set.append(vector)

        #print(c)
        c = c - 1
    counter = collections.Counter(labels)
    #print(counter)
    return feature_set, labels


In [ ]:
#predict labels for raw data
def knnlabel(d):
    x, y = make_datasetknnbase(d)

    neigh = svm.LinearSVC()
    neigh.fit(x, y)

    #x2, y2 ,datafiles = make_datasetknn(d)
    x2,filenum = make_datasetknn(d)


    label=neigh.predict(x2)
    #print(label)

    #here
    counter = collections.Counter(label)
    #print(counter)
    import os
    for i in range(filenum):
      os.rename('/content/semisets/'+str(i)+'.txt','/content/semisets/'+str(i)+'lab#'+str(label[i])+'.txt')
    return x2, label

#knnlabel(d)

In [ ]:
def knn(d):
    x, y = knnlabel(d)
    x1, y1 = make_datasetknnbase(d)
    x.extend(x1)
    y = np.concatenate((y,y1))
    counter = collections.Counter(y1)
    print(counter)



    X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)


    #final = ComplementNB(norm=True)
    #final = MultinomialNB(alpha=0.001)
    final = svm.LinearSVC(C=10.0,max_iter = 7000, random_state =42)
    #final = VotingClassifier(
    #estimators=[('cnb', n), ('mnb', n2),('svm',n1)], voting='hard')
    final.fit(X_train, y_train)


    predictlabel=final.predict(X_test)
    #for i in range(len(y_test)):
        #print("Original: " + str(y_test[i]))
        #print("Predicted: " + str(predictlabel[i]))
    #counter = collections.Counter(predictlabel)
    #print(counter)

#print(neigh.predict([[1, 0, 0, 0]]))

    score =final.score(X_test,y_test)

    from sklearn.metrics import recall_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import confusion_matrix


    print(score ,' Accuracy')
    print(recall_score(y_test,predictlabel, average='macro'),' Recall')
    print(precision_score(y_test,predictlabel, average='macro'),' Precision')
    print(f1_score(y_test,predictlabel, average='macro'),' F1_Measure')
    #tn, fp, fn, tp = confusion_matrix(y_test, predictlabel).ravel()
    #print (tn +" "+fp+" "+fn+" "+tp)

    return score


d = make_di(4000)
print(d)
knn(d)

[('pakistan', 418), ('water', 262), ('league', 254), ('dams', 242), ('please', 192), ('india', 192), ('today', 182), ('one', 164), ('country', 150), ('indian', 144), ('pakistani', 144), ('people', 142), ('us', 138), ('every', 138), ('dam', 138), ('required', 134), ('minister', 134), ('khan', 128), ('contact', 126), ('save', 120), ('money', 120), ('like', 114), ('justice', 114), ('islamabad', 114), ('group', 110), ('players', 110), ('time', 108), ('imran', 108), ('want', 104), ('take', 102), ('rights', 96), ('budget', 96), ('via', 96), ('day', 94), ('know', 94), ('th', 94), ('make', 92), ('pm', 92), ('new', 92), ('human', 92), ('voice', 92), ('going', 90), ('need', 90), ('blood', 90), ('state', 90), ('court', 90), ('must', 90), ('government', 88), ('rs', 88), ('arrested', 88), ('prime', 86), ('spot', 84), ('gulalai', 84), ('life', 84), ('get', 82), ('live', 78), ('see', 78), ('hospital', 78), ('cricket', 78), ('go', 76), ('way', 76), ('also', 74), ('fixers', 74), ('future', 74), ('let',

0.9099447513812154

In [ ]:
import shutil
shutil.rmtree('/content/semisets')
shutil.rmtree('/content/processed_data')
!mkdir semisets
!mkdir processed_data
#preprocess()


# Old code



In [ ]:
#make dataset from semi-supervised labelled data

def make_datasetknnsemi(dictionary):
    knnlabel(dictionary)
    direc = "/content/new_data/"
    files = os.listdir(direc)
    emails = [direc + email for email in files]
    feature_set = []
    labels = []
    c = len(emails)

    for email in emails:
        f = open(email ,encoding="utf8")
        words = f.read().split(' ')
        vector=[]
        singlevec=[]
        for featureword in dictionary:
            w=featureword[0]

            if w in words:
                vector.append(1)
            else:
                vector.append(0)
        #vector.append(singlevec)



        if "politics" in email:
            labels.append(0)
            feature_set.append(vector)
        if "fun" in email:
            labels.append(1)
            feature_set.append(vector)
        if "tech" in email:
            labels.append(2)
            feature_set.append(vector)
        if "curr" in email:
            labels.append(3)
            feature_set.append(vector)
        if "enter" in email:
            labels.append(4)
            feature_set.append(vector)
        if "other" in email:
            labels.append(5)
            feature_set.append(vector)

        #print(c)
        c = c - 1
    return feature_set, labels


In [ ]:
import os
from collections import Counter
import numpy as np
def make_datasetknnsemi():
    direc = "/content/new_basic_human/"
    files = os.listdir(direc)
    emails = [direc + email for email in files]
    feature_set = []
    labels = []
    c = len(emails)
    words = []

    for email in emails:
        f = open(email ,encoding="utf8")
        words.append(f.read())
        vector=[]

        if "politics" in email:
            labels.append('Politics')
        if "fun" in email:
            labels.append("Fun talk")
        if "tech" in email:
            labels.append("Technology")
        if "curr" in email:
            labels.append("Current Events")
        if "enter" in email:
            labels.append("Entertainment")
        if "other" in email:
            labels.append("Other")
    return words, labels

a,b = make_datasetknnsemi()
A = np.vstack([a,b])
print(A)

[['situation ofinis worsen day day  s int' 'coffee habit job taught life'
  'luv u   indeed allah best planner  luv ur words' ...
  'happens  elect honest person  pm shareefs bhuttos looted pakistan bought prope'
  'tayyab bukhari eng deptt name teacher assaulted student saw making video'
  'kanal house available sale block b phase dha lahore']
 ['Other' 'Fun talk' 'Current Events' ... 'Politics' 'Other' 'Other']]


In [ ]:
def preprocess():
  direc = '/content/labelled_data/'
  files = os.listdir(direc)
  emails = [direc + email for email in files]
  stop_words = set(stopwords.words('english'))
  fileno = 0

  for email in emails:
    words = []
    f = open(email, encoding="utf8")
    f_name = os.path.basename(f.name)
    blob = f.read()
    words += blob.split(" ")

    for i in reversed(range(len(words))):
        words[i] = words[i].lower()
        if words[i] in stop_words or not words[i].isalpha() or len(words[i]) < 2:
            words.remove(words[i])

    f2 = open('/content/processed_data/'+f_name,'w')
    for word in words:
      f2.write(word+' ')
    fileno = fileno + 1

In [ ]:
#def check_similarity(feature_word,words):
  #for word in words:
    #res = SequenceMatcher(None, feature_word,word).ratio()
    #if(res>=0.85):
      #return True
  #return False

In [ ]:
d = make_di(4000)
print(d)
x,y = knnlabel(d)
counter = collections.Counter(y)
print(counter)